### Load fantasy premier league data

In [20]:
import requests
import json
import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
from tqdm import tqdm
import os

### Scrape historical data from Fantasy Overlord

In [ ]:
for i in range(5,37):
    url = 'https://fantasyoverlord.com/FPL/History?file=FPL16-GW{}'.format(i)

### Scrape data from Fantasy Premier League API

In [29]:
def load_json(url):
    response = requests.get(url)
    return json.loads(response.content.decode('utf-8'))


def load_player_gameweek():
    gameweek_dfs = []
    for player_id in tqdm(player_summary_df['id'].unique()):
        url = "https://fantasy.premierleague.com/drf/element-summary/" + str(player_id)
        response = requests.get(url)
        gameweek_data = json.loads(response.content.decode('utf-8'))['history']
        gameweek_dfs.append(pd.DataFrame(gameweek_data))
        
    gameweek_df = pd.concat(gameweek_dfs)
    gameweek_df.to_csv('player_gameweeks.csv',index=False)

    
if os.path.exists('player_summaries.csv'):
    player_summary_df = pd.read_csv('player_summaries.csv')
else:
    player_summary_df = pd.DataFrame(load_json('https://fantasy.premierleague.com/drf/elements/'))
    player_summary_df.to_csv('player_summaries.csv',index=False)

    
if os.path.exists('player_gameweeks.csv'):
    player_gameweek_df = pd.read_csv('player_gameweeks.csv')
else:
    load_player_gameweek()
    player_gameweek_df = pd.read_csv('player_gameweeks.csv')
    
player_team_df = pd.DataFrame(load_json('https://fantasy.premierleague.com/drf/teams'))
player_team_df.head()

# https://fantasy.premierleague.com/drf/fixtures
fixture_df = pd.DataFrame(load_json('https://fantasy.premierleague.com/drf/fixtures'))
player_summary_df.head()

,Unnamed: 0,Unnamed: 0.1,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,ea_index,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,id,in_dreamteam,influence,loaned_in,loaned_out,loans_in,loans_out,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,recent_assists,recent_goals_scored,recent_bps,recent_minutes,recent_selected
0,0,0,0,0,56,75.0,100.0,1,48844,0,0,-2,2,0.0,0,0,1,2.3,1.0,1,David,1.0,3,0,4.7,1,False,46.0,0,0,0,0,290,Rib Injury - 75% chance of playing,2018-04-29T20:01:18Z,48,0,0,0,48844.jpg,2.8,0,6,Ospina,0.2,False,13.0,d,1,3,0.0,11,3329,136,9865,98,0.2,2.3,Ospina,0,0.0,0.0,50.0,270.0,62371.0
1,1,1,0,7,576,100.0,100.0,10,11334,0,0,-2,2,0.0,1,0,1,2.6,0.8,0,Petr,0.8,45,0,65.4,2,False,657.0,0,0,0,0,2859,NaN,2018-04-22T12:01:20Z,53,0,0,1,11334.jpg,3.6,0,81,Cech,4.5,False,33.0,a,1,3,0.0,115,179945,1254,290084,2012,0.2,21.7,Cech,1,0.0,0.0,32.0,180.0,1344224.0
2,2,2,0,0,0,0.0,0.0,0,98980,0,0,0,0,0.0,0,0,1,0.0,0.0,0,Damian Emiliano,0.0,0,0,0.0,3,False,0.0,0,0,0,0,0,Season-long loan to Getafe,2017-08-03T08:35:02Z,40,0,0,0,98980.jpg,0.0,0,0,Martinez,0.6,False,26.0,u,1,3,0.0,0,1499,4,29126,81,0.0,0.0,Martinez,0,0.0,0.0,0.0,0.0,175289.0
3,3,3,0,16,556,100.0,100.0,9,51507,0,0,0,0,123.8,3,0,2,2.0,0.5,0,Laurent,0.5,32,2,95.5,4,False,692.0,0,0,0,0,2225,NaN,2018-03-15T22:01:16Z,60,0,0,0,51507.jpg,4.0,0,0,Koscielny,1.7,False,6.0,a,1,3,137.0,100,205717,573,189798,979,0.1,16.7,Koscielny,4,0.0,0.0,18.0,90.0,503976.0
4,4,4,0,2,77,100.0,100.0,0,17127,0,0,-3,3,3.0,1,0,2,1.0,0.0,0,Per,0.0,7,1,15.7,5,False,115.4,0,0,0,0,351,NaN,2017-11-18T12:31:16Z,47,0,0,0,17127.jpg,3.0,0,0,Mertesacker,0.5,False,4.0,a,1,3,38.0,15,31447,63,46287,82,0.0,3.2,Mertesacker,0,0.0,0.0,0.0,0.0,135805.0


In [30]:
for player_id in tqdm(player_summary_df['id'].unique()):
    recent_gameweeks_index = player_gameweek_df['round'].max() - 5
    temp_df = player_gameweek_df[(player_gameweek_df['round'] >= recent_gameweeks_index) &
                                (player_gameweek_df['element'] == player_id)]

    columns = ['total_points','assists','goals_scored','bps','minutes','selected']
    for column in columns:
        new_column_name = 'recent_' + column
        new_column_value = int(temp_df[[column]].sum())
        player_summary_df.loc[player_summary_df['id'] == player_id, new_column_name ] = new_column_value

player_summary_df

100%|██████████| 644/644 [02:14<00:00,  4.79it/s]


,Unnamed: 0,Unnamed: 0.1,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,ea_index,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,id,in_dreamteam,influence,loaned_in,loaned_out,loans_in,loans_out,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,recent_assists,recent_goals_scored,recent_bps,recent_minutes,recent_selected,recent_total_points
0,0,0,0,0,56,75.0,100.0,1,48844,0,0,-2,2,0.0,0,0,1,2.3,1.0,1,David,1.0,3,0,4.7,1,False,46.0,0,0,0,0,290,Rib Injury - 75% chance of playing,2018-04-29T20:01:18Z,48,0,0,0,48844.jpg,2.8,0,6,Ospina,0.2,False,13.0,d,1,3,0.0,11,3329,136,9865,98,0.2,2.3,Ospina,0,0.0,0.0,50.0,270.0,62371.0,10.0
1,1,1,0,7,576,100.0,100.0,10,11334,0,0,-2,2,0.0,1,0,1,2.6,0.8,0,Petr,0.8,45,0,65.4,2,False,657.0,0,0,0,0,2859,NaN,2018-04-22T12:01:20Z,53,0,0,1,11334.jpg,3.6,0,81,Cech,4.5,False,33.0,a,1,3,0.0,115,179945,1254,290084,2012,0.2,21.7,Cech,1,0.0,0.0,32.0,180.0,1344224.0,3.0
2,2,2,0,0,0,0.0,0.0,0,98980,0,0,0,0,0.0,0,0,1,0.0,0.0,0,Damian Emiliano,0.0,0,0,0.0,3,False,0.0,0,0,0,0,0,Season-long loan to Getafe,2017-08-03T08:35:02Z,40,0,0,0,98980.jpg,0.0,0,0,Martinez,0.6,False,26.0,u,1,3,0.0,0,1499,4,29126,81,0.0,0.0,Martinez,0,0.0,0.0,0.0,0.0,175289.0,0.0
3,3,3,0,16,556,100.0,100.0,9,51507,0,0,0,0,123.8,3,0,2,2.0,0.5,0,Laurent,0.5,32,2,95.5,4,False,692.0,0,0,0,0,2225,NaN,2018-03-15T22:01:16Z,60,0,0,0,51507.jpg,4.0,0,0,Koscielny,1.7,False,6.0,a,1,3,137.0,100,205717,573,189798,979,0.1,16.7,Koscielny,4,0.0,0.0,18.0,90.0,503976.0,2.0
4,4,4,0,2,77,100.0,100.0,0,17127,0,0,-3,3,3.0,1,0,2,1.0,0.0,0,Per,0.0,7,1,15.7,5,False,115.4,0,0,0,0,351,NaN,2017-11-18T12:31:16Z,47,0,0,0,17127.jpg,3.0,0,0,Mertesacker,0.5,False,4.0,a,1,3,38.0,15,31447,63,46287,82,0.0,3.2,Mertesacker,0,0.0,0.0,0.0,0.0,135805.0,0.0
5,5,5,0,0,0,0.0,0.0,0,158074,0,0,0,0,0.0,0,0,2,0.0,0.0,0,Gabriel Armando,0.0,0,0,0.0,6,False,0.0,0,0,0,0,0,Joined Valencia on 18/8,2017-07-12T10:31:11Z,50,0,0,0,158074.jpg,0.0,0,0,de Abreu,0.1,False,5.0,u,1,3,0.0,0,365,3,2354,5,0.0,0.0,Gabriel,0,0.0,0.0,0.0,0.0,14481.0,0.0
6,6,6,2,9,580,100.0,100.0,10,98745,0,0,-1,1,534.8,1,0,2,3.6,1.3,3,Héctor,1.3,44,2,150.8,7,False,500.0,0,0,0,0,2870,NaN,2018-03-07T19:31:17Z,59,0,0,0,98745.jpg,3.4,0,0,Bellerín,9.9,False,24.0,a,1,3,472.0,112,719293,6041,954184,2553,0.2,19.0,Bellerín,5,0.0,0.0,70.0,343.0,2936518.0,11.0
7,7,7,0,0,0,0.0,0.0,0,80254,0,0,0,0,0.0,0,0,2,0.0,0.0,0,Carl,0.0,0,0,0.0,9,False,0.0,0,0,0,0,0,Season-long loan to Birmingham,2017-08-22T14:57:50Z,45,0,0,0,80254.jpg,0.0,0,0,Jenkinson,0.2,False,25.0,u,1,3,0.0,0,747,1,13666,7,0.0,0.0,Jenkinson,0,0.0,0.0,0.0,0.0,44948.0,0.0
8,8,8,2,15,601,100.0,100.0,10,38411,1,-1,2,-2,237.9,3,0,2,6.6,2.8,1,Nacho,2.8,29,5,123.6,10,False,690.8,0,0,0,0,2219,NaN,2018-02-25T19:31:19Z,57,0,0,0,38411.jpg,4.8,0,0,Monreal,8.4,False,18.0,a,1,3,305.0,129,1575703,23578,1431768,5217,0.5,23.0,Monreal,3,0.0,1.0,58.0,296.0,2165677.0,17.0
9,9,9,1,0,153,100.0,100.0,2,156074,0,0,-6,6,37.6,0,0,2,2.0,0.5,0,Rob,0.5,14,0,22.4,11,False,163.0,0,0,0,0,639,NaN,2017-11-05T19:31:15Z,44,0,0,0,156074.jpg,2.0,0,0,Holding,0.6,False,16.0,a,1,3,24.0,20,28539,156,110594,202,0.1,4.5,Holding,2,0.0,0.0,23.0,106.0,189934.0,2.0


In [47]:
player_summary_df.to_csv('player_summaries.csv',index=False)

player_summary_df.sort_values(by=['recent_total_points'], ascending=False)\
    [['web_name', 'recent_total_points', 'recent_goals_scored', 'recent_minutes', 'recent_assists']]
    
player_summary_df[player_summary_df['web_name'] == 'Rashford'][['minutes', 'total_points']]

def calc_points_per_minute(minutes, points):
    if minutes == 0 or points == 0:
        return 0
    return minutes/ points

player_summary_df['minutes_per_point'] = player_summary_df.apply(\
                            lambda row: calc_points_per_minute(row['minutes'], row['total_points']), axis=1) 

player_summary_df[(player_summary_df['minutes_per_point'] > 0) & (player_summary_df['minutes'] > 1400)]\
    .sort_values(by=['minutes_per_point'])\
    [['web_name', 'minutes_per_point','recent_total_points', 'recent_goals_scored', 'recent_minutes', 'recent_assists']]
    

,web_name,minutes_per_point,recent_total_points,recent_goals_scored,recent_minutes,recent_assists
307,Salah,9.388316,54.0,7.0,443.0,1.0
326,Sterling,10.780269,44.0,3.0,403.0,6.0
336,Agüero,11.597633,0.0,0.0,14.0,0.0
368,Martial,12.048780,5.0,0.0,148.0,0.0
337,Jesus,12.452991,33.0,4.0,352.0,2.0
516,Son,12.864706,16.0,0.0,328.0,2.0
133,Hazard,13.293413,18.0,1.0,364.0,1.0
334,Sané,13.490566,22.0,2.0,333.0,1.0
14,Ramsey,13.603306,15.0,1.0,180.0,1.0
367,Lingard,13.806452,17.0,0.0,329.0,2.0
